# グラフ作成　2022/6/6

画像を畳み込みして6枚取得  
一枚に加算  
2次元配列を1次元に変換  
配列を小さい順にソート  
大きいほうからノード数分の数値のインデックスを取得  
取得したインデックスから元の座標を計算  
座標から6つの画素値を取得  
座標からノード間の距離を計算  


In [1]:
#必要なパッケージをインポート
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import urllib.request
import dgl
from dgl.data import DGLDataset
import os
import itertools
import time


%matplotlib inline

/home/echigo/anaconda3/envs/DGL2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#CIFAR-10の読み込み
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = torchvision.datasets.CIFAR10(root = '../data', train = True, download = True, transform = transform)
testset = torchvision.datasets.CIFAR10(root='../data',train=False,download=True,transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#畳み込み結果を取得するためのネットワーククラス
class Conv_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.pool = nn.MaxPool2d(2,2)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        return x


class Dence_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x


class Nested_net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_net = Conv_layer()
        self.Dence_net = Dence_layer()

    def forward(self,x):
        x = self.conv_net(x)
        x = x.view(-1, 16 * 5 * 5)
        x = self.Dence_net(x)

        return x

In [4]:
#処理を高速化するためにGPUを使用するための設定。モデルをGPUへ
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Nested_net()
model.to(device)

#学習済みモデルをロード
path = '../models/Nest_model.pth'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [5]:
def get_distance(x,y): #距離計算用関数
    hdist = ((x[0] - y[0])**2 + (x[1] - y[1])**2)
    #hdist = torch.tensor(hdist)
    hdist = hdist.clone().detach()
    dist = torch.sqrt(hdist)

    return dist


def return_two_list(node_num):
    taikaku = torch.full((node_num,node_num),fill_value=1.)
    for i in range(node_num):
        taikaku[i][i] = 0.
    src_ids = []
    dst_ids = []
    for i in range(node_num):
        for j in range(i,node_num):
            if taikaku[i][j] != 0:
                src_ids.append(i)
                dst_ids.append(j)
                src_ids.append(j)
                dst_ids.append(i)
    tensor_src = torch.tensor(src_ids)
    tensor_dst = torch.tensor(dst_ids)
    return tensor_src,tensor_dst

    
def zscore(x,axis = 0):
    stddev, mean = torch.std_mean(x,dim=axis,unbiased=True)
    result = (x - mean)/stddev
    return result

1.ノード特徴量：画素値　エッジ特徴量：距離  
2.ノード特徴量：距離　　エッジ特徴量：なし

In [6]:
#設定
pool = nn.MaxPool2d(2,2)
train_data_number = 'full' #0~5000 or full
num_node = 5
graph_type = 1 #1(n:feat value, e:distance) or 2(n:distance, e:none)
mode = None
save1_file_name = f'{mode}_6feat_dist_{num_node}.dgl'
save2_file_name = f'{mode}_dist_{num_node}.dgl'

In [13]:
graphs = []
labels = []
class_counter = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0} #カウント用辞書
train_class_counter = {0:train_data_number,1:train_data_number,2:train_data_number,3:train_data_number,4:train_data_number,
                       5:train_data_number,6:train_data_number,7:train_data_number,8:train_data_number,9:train_data_number}
src,dst = return_two_list(num_node)

#train start
mode = 'train'
if type(train_data_number) is int:
    check = True
    flag = False
else:
    check = False
for image,label in tqdm(trainset):
    if check == True:
        if class_counter == train_class_counter:
            break
        if class_counter[label] == train_data_number:
            continue
        else:
            class_counter[label] += 1
    image = image.to(device)
    image = image.unsqueeze(0)
    features_maps = F.relu(model.conv_net.conv1(image))#BCSS
    image_size = features_maps.shape[2]
    #print(features_maps.shape)
    features_maps = features_maps.permute(1,2,3,0)#CSSB
    #print(features_maps.shape)
    synthetic_map = 0
    for i in features_maps:#SSC
        synthetic_map += i
    #print(synthetic_map.shape)
    synthetic_map = torch.squeeze(synthetic_map)
    zero2one_map = synthetic_map/torch.max(synthetic_map)
    onedim = zero2one_map.reshape(image_size * image_size)
    #print(onedim.shape)
    sort_onedim,sort_index = torch.sort(onedim)
    #print(sort_onedim[-5:],sort_index[-5:])
    ori_index = torch.empty(num_node,2,dtype=torch.float32,device=device)
    for i in range(num_node):
        x = sort_index[-num_node:][i] // image_size
        y = sort_index[-num_node:][i] - x * image_size
        ori_index[i][0] = x
        ori_index[i][1] = y
    print(ori_index)
    distance = []
    for i in range(num_node-1):
        for j in range(i+1,num_node):
            distance.append(get_distance(ori_index[i],ori_index[j]))
    print(distance)
    break
    

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_18411/580188855.py:42: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x = sort_index[-num_node:][i] // image_size
  0%|          | 0/50000 [00:00<?, ?it/s]

tensor([[13., 23.],
        [17., 23.],
        [13., 22.],
        [16.,  8.],
        [15.,  8.]], device='cuda:0')
[tensor(4., device='cuda:0'), tensor(1., device='cuda:0'), tensor(15.2971, device='cuda:0'), tensor(15.1327, device='cuda:0'), tensor(4.1231, device='cuda:0'), tensor(15.0333, device='cuda:0'), tensor(15.1327, device='cuda:0'), tensor(14.3178, device='cuda:0'), tensor(14.1421, device='cuda:0'), tensor(1., device='cuda:0')]


In [ ]:
graphs = []
labels = []
class_counter = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0} #カウント用辞書
train_class_counter = {0:train_count,1:train_count,2:train_count,3:train_count,4:train_count,5:train_count,6:train_count,7:train_count,8:train_count,9:train_count} #トレーニング用比較辞書
test_class_counter = {0:200,1:200,2:200,3:200,4:200,5:200,6:200,7:200,8:200,9:200} #テスト用比較辞書
src,dst = return_two_list(num_node)

for img_idx in tqdm(range(50000)):
    image, label = trainset[img_idx]
    '''
    if class_counter == train_class_counter: #クラスカウントチェック
        break
    if class_counter[label] == train_count:
        continue
    else:
        class_counter[label] += 1
    '''
    class_counter[label] += 1
    image = image.to(device)

    image = image.unsqueeze(0) #バッチ用次元を追加
    features_maps = F.relu(model.conv_net.conv1(image)) #特徴マップの生成(torch.Size([1, 6, 14, 14]))
    image_size = features_maps.shape[2]
    #print(features_maps.shape)
    features_maps = features_maps.permute(1,2,3,0) #torch.Size([6, 14, 14, 1])

    synthetic_map = 0 #合成した特徴マップ
    for i in range(len(features_maps)): #6枚を1枚へ合成
        synthetic_map += features_maps[i]
    
    #zero2one_map = synthetic_map / synthetic_map.max() #合成したものを0-1に変換
    zero2one_map = zscore(synthetic_map)

    onedim = zero2one_map.reshape(image_size*image_size) #0-1画像を14*14の一次元配列に変換
    
    sort_onedim, sort_index = torch.sort(onedim) #小さい順に並べ替え

    ori_index = torch.empty(num_node,2,dtype=torch.float32,device=device) #元の2次元配列においての座標リスト
    for i in range(num_node):
        x = sort_index[-num_node:][i] // image_size
        y = sort_index[-num_node:][i] - x * image_size
        ori_index[i][0] = x
        ori_index[i][1] = y

    '''
    node_data = torch.empty([num_node,2],device=device) #座標のみ
    node_data = ori_index
    '''
    

    std_index = ori_index / image_size
    node_data = torch.empty([num_node,8],device=device) #各画素値6つ + 座標2つ
    for i in range(num_node):
        six_f = torch.empty(8,device = device)
        for j in range(6): #画素値代入
            x = int(ori_index[i][0].item())
            y = int(ori_index[i][1].item())
            six_f[j] = features_maps[j][x][y]
        for ind,j in enumerate(range(6,8)): #座標代入
            six_f[j] = std_index[i][ind]
        node_data[i] = six_f
    
    
    edge_data = torch.empty([num_node * (num_node -1),1],device=device) #エッジデータの作成 10エッジx双方向 = 20個
    for i,comb in enumerate(itertools.combinations(ori_index,2)):
        d = get_distance(comb[0],comb[1])
        edge_data[i*2] = d
        edge_data[i*2 + 1] = d

    edge_data = zscore(edge_data) #距離の標準化
    

    #グラフ作成
    g = dgl.graph((src,dst),num_nodes=num_node,device=device)
    g.edata['distance'] = edge_data #エッジ特徴の代入
    g.ndata['feat value'] = node_data #ノード特徴の代入

    #保存用リストに追加
    graphs.append(g)
    labels.append(label)
print(len(graphs))
print(len(labels))
print(class_counter)
print(f'ノード数: {num_node}')
output_labels = {'label':torch.tensor(labels)}
path = f'../data/MyDataset/train_graphs_six_f_in_pos_{num_node}_std_full.dgl'
dgl.save_graphs(path,g_list=graphs,labels=output_labels)
#テスト用グラフ作成本番用セル。ノードデータは6枚の特徴マップの対応する画素値

pool = nn.MaxPool2d(2,2)  #MaxPooling層の設定
graphs = []
labels = []
#num_node = 40
class_counter = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0} #カウント用辞書
train_class_counter = {0:1000,1:1000,2:1000,3:1000,4:1000,5:1000,6:1000,7:1000,8:1000,9:1000} #トレーニング用比較辞書
test_class_counter = {0:200,1:200,2:200,3:200,4:200,5:200,6:200,7:200,8:200,9:200} #テスト用比較辞書

src,dst = return_two_list(num_node)
for img_idx in tqdm(range(10000)):
    image, label = testset[img_idx]

    '''
    if class_counter == test_class_counter: #クラスカウントチェック
        break
    if class_counter[label] == 200:
        continue
    else:
        class_counter[label] += 1
    '''
    class_counter[label] += 1
    image = image.to(device)

    image = image.unsqueeze(0) #バッチ用次元を追加
    features_maps = F.relu(model.conv_net.conv1(image)) #特徴マップの生成(torch.Size([1, 6, 14, 14]))
    image_size = features_maps.shape[2]
    #print(features_maps.shape)
    features_maps = features_maps.permute(1,2,3,0) #torch.Size([6, 14, 14, 1])

    synthetic_map = 0 #合成した特徴マップ
    for i in range(len(features_maps)): #6枚を1枚へ合成
        synthetic_map += features_maps[i]
    
    #zero2one_map = synthetic_map / synthetic_map.max() #合成したものを0-1に変換
    zero2one_map = zscore(synthetic_map)

    onedim = zero2one_map.reshape(image_size*image_size) #0-1画像を14*14の一次元配列に変換
    
    sort_onedim, sort_index = torch.sort(onedim) #小さい順に並べ替え

    ori_index = torch.empty(num_node,2,dtype=torch.float32,device=device) #元の2次元配列においての座標リスト
    for i in range(num_node):
        x = sort_index[-num_node:][i] // image_size
        y = sort_index[-num_node:][i] - x * image_size
        ori_index[i][0] = x
        ori_index[i][1] = y
    
    '''
    node_data = torch.empty([num_node,2],device=device)
    node_data = ori_index
    '''
    
    std_index = ori_index / image_size
    node_data = torch.empty([num_node,8],device=device)
    for i in range(num_node):
        six_f = torch.empty(8,device = device)
        for j in range(6):
            x = int(ori_index[i][0].item())
            y = int(ori_index[i][1].item())
            six_f[j] = features_maps[j][x][y]
        for ind,j in enumerate(range(6,8)): #座標代入
            six_f[j] = std_index[i][ind]
        node_data[i] = six_f
    
    
    edge_data = torch.empty([num_node * (num_node - 1),1],device=device) #エッジデータの作成 10エッジx双方向 = 20個
    for i,comb in enumerate(itertools.combinations(ori_index,2)):
        d = get_distance(comb[0],comb[1])
        edge_data[i*2] = d
        edge_data[i*2 + 1] = d

    edge_data = zscore(edge_data)

    #グラフ作成
    g = dgl.graph((src,dst),num_nodes=num_node,device=device)
    g.edata['distance'] = edge_data #エッジ特徴の代入
    g.ndata['feat value'] = node_data #ノード特徴の代入

    #保存用リストに追加
    graphs.append(g)
    labels.append(label)
print(len(graphs))
print(len(labels))
print(class_counter)
print(f'ノード数: {num_node}')
output_labels = {'label':torch.tensor(labels)}
path = f'../data/MyDataset/test_graphs_six_f_in_pos_{num_node}_std_full.dgl'
dgl.save_graphs(path,g_list=graphs,labels=output_labels)

  0%|          | 0/50000 [00:00<?, ?it/s]/tmp/ipykernel_3776/50535596.py:45: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x = sort_index[-num_node:][i] // image_size
  5%|▌         | 2602/50000 [04:28<13:35:07,  1.03s/it]

: 

: 